# CIA Factbook Data Analysis

The data for this work is taken from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:

* **population** — the global population.
* **population_growth** — the annual population growth rate, as a percentage.
* **area** — the total land and water area.

## Introduction

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Overview of the Data

In [3]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Summary Statistics

Calculation for min/max of popupation and population growth

In [4]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


Columns with MIN/Max population need to be investigated more, as it is very strange the country with 0 population.

## Exploring Outliers 

1. Minimal population

In [5]:
%%sql
SELECT *
  FROM facts
 WHERE population = 0;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


2. Maximal population

In [6]:
%%sql
SELECT *
  FROM facts
 WHERE population = 7256490011;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


**Antarctica** and **World** cannot be considered as countries and will be excluded from the next investigations of the data.

Below updated summary statistics without Antarctica and World.



In [7]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts
 WHERE name <> 'Antarctica' AND name <> 'World';

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
48,1367485388,0.0,4.02


## Exploring Average Population and Area

In [8]:
%%sql
SELECT AVG(population), AVG(area)
  FROM facts
 WHERE name <> 'Antarctica' AND name <> 'World';

 * sqlite:///factbook.db
Done.


AVG(population),AVG(area)
32377011.0125,555093.546184739


## Finding Densly Populated Countries

The following logic wiil be used: 

* countries that are above average population
* countries that have area below average

In [9]:
%%sql
SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name <> 'Antarctica' AND name <> 'World')
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name <> 'Antarctica' AND name <> 'World');

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


## Densly populated countries. Comparasion on 1 km2.

We will compare our previous results, but now we will find out densly populated countries by finding amount people on 1 km2.

In [10]:
%%sql
SELECT *, population/area AS pop_per_1km2
  FROM facts
 WHERE name <> 'Antarctica' AND name <> 'World'
 ORDER BY pop_per_1km2 DESC
 LIMIT 10;
 

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,pop_per_1km2
205,mc,Macau,28,28,0,592731,0.8,8.88,4.22,3.37,21168
117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83,15267
156,sn,Singapore,697,687,10,5674472,1.89,8.27,3.43,14.05,8141
204,hk,Hong Kong,1108,1073,35,7141106,0.38,9.23,7.07,1.68,6445
251,gz,Gaza Strip,360,360,0,1869055,2.81,31.11,3.04,0.0,5191
233,gi,Gibraltar,6,6,0,29258,0.24,14.08,8.37,3.28,4876
13,ba,Bahrain,760,760,0,1346613,2.41,13.66,2.69,13.09,1771
108,mv,Maldives,298,298,0,393253,0.08,15.75,3.89,12.68,1319
110,mt,Malta,316,316,0,413965,0.31,10.18,9.09,1.98,1310
227,bd,Bermuda,54,54,0,70196,0.5,11.33,8.23,1.88,1299


The result includes small countries with area range 2-1073 square kilometres and high level of population.

## Finding birth_death ratio

We are going to calculate birth/death ratio and find out top 10 countries with positive and negative ratio.

1.  Top 10 positive birth_death ratio

In [11]:
%%sql
SELECT *, ROUND(birth_rate/death_rate,2) AS birth_death_ratio
  FROM facts
 ORDER BY birth_death_ratio DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,birth_death_ratio
251,gz,Gaza Strip,360,360,0,1869055,2.81,31.11,3.04,0.0,10.23
93,ku,Kuwait,17818,17818,0,2788534,1.62,19.91,2.18,1.58,9.13
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62,8.34
184,ae,United Arab Emirates,83600,83600,0,5779760,2.58,15.43,1.97,12.36,7.83
131,mu,Oman,309500,309500,0,3286936,2.07,24.44,3.36,0.43,7.27
86,jo,Jordan,89342,88802,540,8117564,0.83,25.37,3.79,13.24,6.69
159,bp,Solomon Islands,28896,27986,910,622469,2.02,25.77,3.85,1.75,6.69
254,we,West Bank,5860,5640,220,2785366,1.95,22.99,3.5,0.0,6.57
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39,6.43
189,nh,Vanuatu,12189,12189,0,272264,1.95,25.04,4.09,1.47,6.12


From our result we see that large amount of countries with high birth_death ratio from Middle East region.

2. Top 20 countries with negative birth_death ratio


In [12]:
%%sql
SELECT *, ROUND(birth_rate/death_rate,2) AS birth_death_ratio       
  FROM facts
 WHERE birth_rate <> 'None' AND death_rate <> 'None'
 ORDER BY birth_death_ratio
 LIMIT 20;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,birth_death_ratio
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29,0.62
153,ri,Serbia,77474,77474,0,7176794,0.46,9.08,13.66,0.0,0.66
96,lg,Latvia,64589,62249,2340,1986705,1.06,10.0,14.31,6.26,0.7
102,lh,Lithuania,65300,62680,2620,2884433,1.04,10.1,14.27,6.27,0.71
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33,0.72
117,mn,Monaco,2,2,0,30535,0.12,6.65,9.24,3.83,0.72
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24,0.74
158,si,Slovenia,20273,20151,122,1983412,0.26,8.42,11.37,0.37,0.74
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25,0.74
214,sb,Saint Pierre and Miquelon,242,242,0,5657,1.08,7.42,9.72,8.49,0.76


We increased our catching group to 20 countries and see that almost all countries from the Eastern and Southen Europe + Baltic countries have low birth_death ratio.

## Exploring Eastern Europe

Now we will look deeper in the Eastern Europe and find out which country have the highest birth_death_ratio, which one is the most densly populated, how much water each country have per person.

These are the [countries of Eastern Europe](https://www.worldatlas.com/articles/what-countries-are-in-eastern-europe.html)
- Belarus
- Bulgaria
- Czech Republic
- Hungary
- Poland
- Moldova
- Romania
- Russia
- Slovakia
- Ukraine

**1. Birth_death ratio**

In [13]:
%%sql
SELECT *, ROUND(birth_rate/death_rate,2) AS birth_death_ratio       
  FROM facts
 WHERE name IN ('Belarus','Bulgaria', 'Czech Republic', 'Hungary', 'Poland', 'Moldova', 'Romania', 'Russia', 'Slovakia', 'Ukraine' )
 ORDER BY birth_death_ratio DESC;


 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,birth_death_ratio
157,lo,Slovakia,49035,48105,930,5445027,0.02,9.91,9.74,0.04,1.02
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46,0.96
116,md,Moldova,33851,32891,960,3546847,1.03,12.0,12.59,9.67,0.95
47,ez,Czech Republic,78867,77247,1620,10644842,0.16,9.63,10.34,2.33,0.93
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69,0.85
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7,0.8
142,ro,Romania,238391,229891,8500,21666350,0.3,9.14,11.9,0.24,0.77
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25,0.74
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33,0.72
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29,0.62


We see the ratio is below 1 and only Slovakia have above 1. That means that rest countries have negative tendency to reproduction, on 1 death we have 0.62 - 0.96 births 


**2. Density of population**

In [14]:
%%sql
SELECT *, population/area AS pop_per_1km2
  FROM facts
 WHERE name IN ('Belarus','Bulgaria', 'Czech Republic', 'Hungary', 'Poland', 'Moldova', 'Romania', 'Russia', 'Slovakia', 'Ukraine')
 ORDER BY pop_per_1km2 DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,pop_per_1km2
47,ez,Czech Republic,78867,77247,1620,10644842,0.16,9.63,10.34,2.33,134
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46,123
157,lo,Slovakia,49035,48105,930,5445027,0.02,9.91,9.74,0.04,111
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33,106
116,md,Moldova,33851,32891,960,3546847,1.03,12.0,12.59,9.67,104
142,ro,Romania,238391,229891,8500,21666350,0.3,9.14,11.9,0.24,90
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25,73
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29,64
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7,46
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69,8


Chech Republic is the most densly populated country in the Eastern Europe and Russia, being the largest country in the world by area, have 8 people per 1 km2 (https://en.wikipedia.org/wiki/Russia)

**3. Water bank**

As a reminder water area means:
* area_water - The country's waterarea in square kilometers

In [15]:
%%sql
SELECT *, ROUND(CAST(area_water AS FLOAT)/population*1000,2) AS water_KM2_per_1th_pop
  FROM facts
 WHERE name IN ('Belarus','Bulgaria', 'Czech Republic', 'Hungary', 'Poland', 'Moldova', 'Romania', 'Russia', 'Slovakia', 'Ukraine')
 ORDER BY water_KM2_per_1th_pop DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,water_KM2_per_1th_pop
143,rs,Russia,17098242,16377742,720500,142423773,0.04,11.6,13.69,1.69,5.06
183,up,Ukraine,603550,579330,24220,44429471,0.6,10.72,14.46,2.25,0.55
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7,0.49
142,ro,Romania,238391,229891,8500,21666350,0.3,9.14,11.9,0.24,0.39
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33,0.35
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29,0.33
116,md,Moldova,33851,32891,960,3546847,1.03,12.0,12.59,9.67,0.27
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46,0.22
157,lo,Slovakia,49035,48105,930,5445027,0.02,9.91,9.74,0.04,0.17
47,ez,Czech Republic,78867,77247,1620,10644842,0.16,9.63,10.34,2.33,0.15


The largest water bank have Russia with more than 5km2 of water area per 1 thousand of population.